## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-15-03-53-18 +0000,nypost,‘The View’ co-host Ana Navarro urges Gloria Ga...,https://nypost.com/2025/08/14/entertainment/th...
1,2025-08-15-03-52-46 +0000,nyt,Federal Raids Target D.C. Homeless Camps,https://www.nytimes.com/2025/08/14/us/politics...
2,2025-08-15-03-51-00 +0000,wsj,Attorney General Names DEA Chief to Take Over ...,https://www.wsj.com/us-news/attorney-general-n...
3,2025-08-15-03-44-00 +0000,wsj,Japan’s Economy Records Modest Growth Despite ...,https://www.wsj.com/economy/japans-economy-rec...
4,2025-08-15-03-40-00 +0000,wsj,"China’s Economy Shows Signs Of Slowing, Raisin...",https://www.wsj.com/world/china/chinas-economy...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2364/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,59
149,putin,26
165,will,12
77,year,11
150,summit,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
31,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...,138
203,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...,129
17,2025-08-15-02-38-36 +0000,nypost,Anchorage prepares for Trump-Putin summit as s...,https://nypost.com/2025/08/14/us-news/anchorag...,127
249,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...,124
145,2025-08-14-19-03-21 +0000,cbc,"WATCH | Zelenskyy, Starmer meet ahead of Trump...",https://www.cbc.ca/player/play/9.6867280?cmp=rss,122


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
31,138,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...
82,50,2025-08-14-22-26-00 +0000,wsj,California Democrats are planning their own re...,https://www.wsj.com/politics/policy/gavin-news...
67,48,2025-08-14-22-55-34 +0000,nypost,Long Island mom smashes 14-year-old girl in th...,https://nypost.com/2025/08/14/us-news/long-isl...
60,48,2025-08-14-23-11-41 +0000,startribune,"New details, charges emerge as Vance Boelter i...",https://www.startribune.com/hennepin-county-gr...
188,44,2025-08-14-16-23-09 +0000,nypost,"Toronto film festival reverses course, will ‘e...",https://nypost.com/2025/08/14/world-news/tiff-...
203,38,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...
226,37,2025-08-14-14-14-14 +0000,cbc,"With flight attendant strike looming, union ac...",https://www.cbc.ca/news/business/air-canada-pa...
72,35,2025-08-14-22-48-00 +0000,nypost,Nike co-founder Phil Knight and wife pledge re...,https://nypost.com/2025/08/14/us-news/nike-co-...
185,33,2025-08-14-16-29-11 +0000,nypost,DC cops allowed to alert ICE about illegal imm...,https://nypost.com/2025/08/14/us-news/dc-cops-...
281,29,2025-08-14-09-35-50 +0000,bbc,Watch: Moment car is hit by drone in Russia's ...,https://www.bbc.co.uk/programmes/p0lwvwqq?at_m...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
